In [66]:
import os
import locale
import numpy as np
import pandas as pd
import requests as r
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')

import numpy as np
np.printoptions(precision=2)

In [11]:
def fetch_data(url):
    df = pd.read_html(url)[0]
#     df = pd.read_csv("static/corona.csv", index_col=False)
    if len(df.columns) >= 8:
        df.drop(df.columns.values[-1], axis=1, inplace=True)
        new_col_names = ["Country", "Cases", "NewPatients", "Deaths", "RecentDeaths",
                         "Recovered", "ActivePatients", "CriticalPatients"]
        rename_dict = dict(zip(df.columns.values, new_col_names))
        df = df.rename(columns=rename_dict)
        df["NewPatients"] = df["NewPatients"].apply(lambda x: x is not None and type(x) is not float and '+' in x and atof(x.split('+')[1]))
        df["NewPatients"] = df["NewPatients"].replace(False, 0)
        df.drop(df[df["Country"].apply(lambda x: "Total" in x)].index.values, inplace=True)
        df.fillna(value=0, inplace=True)
        return df
    return {"code": 404}

In [12]:
res = r.get("https://www.worldometers.info/coronavirus/") 
df = fetch_data(res.text)
df.head()

,Country,Cases,NewPatients,Deaths,RecentDeaths,Recovered,ActivePatients,CriticalPatients
0,China,80866,0.0,3213.0,14.0,67762.0,9891,3226.0
1,Italy,24747,0.0,1809.0,0.0,2335.0,20603,1672.0
2,Iran,13938,0.0,724.0,0.0,4590.0,8624,0.0
3,S. Korea,8236,0.0,75.0,0.0,1137.0,7024,59.0
4,Spain,7988,0.0,294.0,0.0,517.0,7177,272.0


In [67]:
workdf = df.sort_values("Cases", ascending=0).head(30)

In [68]:
country, activePatient, recovered, recoveryRate, deathRate = df["Country"], df["ActivePatients"], df["Recovered"], (df["Recovered"] / df["Cases"]) * 100, (df["Deaths"]/ df["Cases"]) * 100

In [80]:
data = {"Country": country,
        "ActivePatient": activePatient,
        "Recovered": recovered,
        "RecoveryRate": recoveryRate.apply(lambda x: round(x)),
        "DeathRate": deathRate.apply(lambda x: round(x))}

In [92]:
work = pd.DataFrame(data, )

TypeError: Index(...) must be called with a collection of some kind, False was passed

In [82]:
work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        158 non-null    object 
 1   ActivePatient  158 non-null    int64  
 2   Recovered      158 non-null    float64
 3   RecoveryRate   158 non-null    int64  
 4   DeathRate      158 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 12.4+ KB


In [83]:
work = work.fillna(value=0)

In [84]:
work.head()

,Country,ActivePatient,Recovered,RecoveryRate,DeathRate
0,China,9891,67762.0,84,4
1,Italy,20603,2335.0,9,7
2,Iran,8624,4590.0,33,5
3,S. Korea,7024,1137.0,14,1
4,Spain,7177,517.0,6,4


In [85]:
fresh = work[work["ActivePatient"] > 0][ work["RecoveryRate"] > 0]

/home/ipd/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [86]:
sort_by_RR = fresh.sort_values("RecoveryRate", ascending=0, ignore_index=True)

In [87]:
sort_by_RR.head()

,Country,ActivePatient,Recovered,RecoveryRate,DeathRate
0,Macao,1,10.0,91,0
1,China,9891,67762.0,84,4
2,Diamond Princess,233,456.0,66,1
3,Hong Kong,64,81.0,54,3
4,Nigeria,1,1.0,50,0


In [88]:
df.iloc[94]

Country             Macao
Cases                  11
NewPatients             0
Deaths                  0
RecentDeaths            0
Recovered              10
ActivePatients          1
CriticalPatients        0
Name: 94, dtype: object

In [89]:
sort_by_RR.sort_values("DeathRate", ascending=0)

,Country,ActivePatient,Recovered,RecoveryRate,DeathRate
45,Philippines,123,5.0,4,9
16,Iraq,88,26.0,21,8
27,Italy,20603,2335.0,9,7
18,Algeria,40,10.0,19,7
44,San Marino,98,4.0,4,6
...,...,...,...,...,...
32,Lithuania,13,1.0,7,0
36,Romania,149,9.0,6,0
38,Afghanistan,20,1.0,5,0
39,North Macedonia,18,1.0,5,0


In [90]:
di = sort_by_RR.head().to_dict()

In [91]:
di.keys()

dict_keys(['Country', 'ActivePatient', 'Recovered', 'RecoveryRate', 'DeathRate'])